# Prática 1 - Inteligência Artificial


1. Implementar o algoritmo A* para resolver o labirinto.
2. Retornar os nós visistados e suas f's (h(n) + g(n)).
3. Definir a Heurística (h(n)). 

---
**Nome**: Melissa Junqueira de Barros Lins

**RA**: 11201920583


---
*Links*

**Medidas de distância**: https://mineracaodedados.wordpress.com/2012/02/27/medidas-de-distancia/






## Labirinto 



In [127]:
#Bibliotecas importadas
from random import shuffle, randrange 
import queue
import sys

#Código
def cria_labirinto(larg = 8, alt = 3):
  vis = [[0] * larg + [1] for _ in range(alt)] + [[1] * (larg + 1)] 
  sem_muros = []

  def quebra_muros(lin, col):
    vis[lin][col] = 1
    d = [(lin, col + 1), (lin, col - 1), (lin - 1, col), (lin + 1, col)]
    shuffle(d)

    for (l, c) in d:
      if vis[l][c] != 1:
        sem_muros.append((lin, col, l, c))
        quebra_muros(l, c)

  quebra_muros(randrange(alt), randrange(larg))
  return(sem_muros)

def desenha_labirinto(lab, larg = 8, alt = 3):
  ver = [['|  '] * larg + ['|'] for _ in range(alt)] + [[]]
  hor = [['+--'] * larg + ['+'] for _ in range(alt + 1)]
  for (l1, c1, l2, c2) in lab:
    if l1 == l2:
      ver[l1][max(c1, c2)] = '   '
    if c1 == c2:
      hor[max(l1, l2)][c1] = '+  '
  for (a, b) in zip(hor, ver):
    print(''.join(a + ['\n'] + b))

labirinto = cria_labirinto()
desenha_labirinto(labirinto)

+--+--+--+--+--+--+--+--+
|           |        |  |
+  +--+--+  +  +  +  +  +
|  |     |  |  |  |     |
+  +  +  +  +  +  +--+  +
|     |  |     |     |  |
+--+--+--+--+--+--+--+--+



## Resolução


- Heurística (h(n)): Distância de Manhattan(City Block)




In [128]:
def calcula_heuristica(ponto_inicial, ponto_final):
  (xi, yi) = ponto_inicial
  (xf, yf) = ponto_final
  return abs(xf - xi) + abs(yf - yi) 

*   Código resposta




In [129]:
def teste_final(posicao,fim) -> bool:
  return posicao == fim

In [130]:
def calculaSucessores(labirinto,larg = 8,alt = 3):
  moves = {}
  for i in range(alt):
    for j in range(larg):
      moves[(i,j)] = []
  for k in range(len(labirinto)):
    x, y, neighborx, neighbory = labirinto[k]
    position = (x, y)
    neighbor = (neighborx, neighbory)
    moves[position].append(neighbor)
    moves[neighbor].append(position)
  return moves

In [131]:
print(calculaSucessores(labirinto))

{(0, 0): [(0, 1), (1, 0)], (0, 1): [(0, 2), (0, 0)], (0, 2): [(0, 3), (0, 1)], (0, 3): [(1, 3), (0, 2)], (0, 4): [(0, 5), (1, 4)], (0, 5): [(1, 5), (0, 6), (0, 4)], (0, 6): [(0, 5), (1, 6)], (0, 7): [(1, 7)], (1, 0): [(0, 0), (2, 0)], (1, 1): [(2, 1), (1, 2)], (1, 2): [(1, 1), (2, 2)], (1, 3): [(2, 3), (0, 3)], (1, 4): [(0, 4), (2, 4)], (1, 5): [(2, 5), (0, 5)], (1, 6): [(0, 6), (1, 7)], (1, 7): [(1, 6), (0, 7), (2, 7)], (2, 0): [(1, 0), (2, 1)], (2, 1): [(2, 0), (1, 1)], (2, 2): [(1, 2)], (2, 3): [(2, 4), (1, 3)], (2, 4): [(1, 4), (2, 3)], (2, 5): [(2, 6), (1, 5)], (2, 6): [(2, 5)], (2, 7): [(1, 7)]}


In [132]:
def a_estrela(labirinto,inicio,objetivo):
  sucessores = calculaSucessores(labirinto)  
  explorados = {inicio: calcula_heuristica(inicio,objetivo) + 1 }
  fila = queue.LifoQueue()
  percorre(sucessores, explorados, fila, inicio, objetivo)
  
  print()
  print("=" * 15)
  print(f'  Nós Abertos  ')
  print("=" * 15)
  for no in explorados:
    print(f'{no} f(n): {explorados[no]}')
  print("=" * 15)
  print()

  caminho = queue.LifoQueue()
  while not fila.empty():  
    caminho.put(fila.get())

  caminhoStr = ""
  while not caminho.empty():  
    caminhoStr += " >> " + str(caminho.get());

  print('Menor Caminho: '+ caminhoStr)

In [133]:
def percorre(sucessores, explorados, fila, atual, objetivo):
  if(teste_final(atual, objetivo)):
    fila.put(atual)
    return
  adjacentes = sucessores.get(atual)
  print(f'Adjacentes de {atual}: {adjacentes}')
  menorCusto = sys.maxsize
  menorAdjacente = None
  for adjacente in adjacentes:
    if(adjacente in explorados):
      continue
    heuristica = calcula_heuristica(adjacente, objetivo)
    custoAdjacente = heuristica + 1;
    print(f'    Custo de {adjacente}: {custoAdjacente} ')
    if(menorCusto > custoAdjacente):
      menorCusto = custoAdjacente
      menorAdjacente = adjacente;

  if(menorAdjacente == None):
    print(f'    O nó {atual} está sem saída.')      
    percorre(sucessores, explorados, fila, fila.get(), objetivo)
  else:
    print(f'    Menor Custo: {menorAdjacente} = {menorCusto}')      
    explorados[menorAdjacente] = custoAdjacente
    fila.put(atual)
    percorre(sucessores, explorados, fila, menorAdjacente, objetivo)
    

In [134]:
inicio = (0,0)
objetivo = (2,7)
a_estrela(labirinto, inicio, objetivo)

Adjacentes de (0, 0): [(0, 1), (1, 0)]
    Custo de (0, 1): 9 
    Custo de (1, 0): 9 
    Menor Custo: (0, 1) = 9
Adjacentes de (0, 1): [(0, 2), (0, 0)]
    Custo de (0, 2): 8 
    Menor Custo: (0, 2) = 8
Adjacentes de (0, 2): [(0, 3), (0, 1)]
    Custo de (0, 3): 7 
    Menor Custo: (0, 3) = 7
Adjacentes de (0, 3): [(1, 3), (0, 2)]
    Custo de (1, 3): 6 
    Menor Custo: (1, 3) = 6
Adjacentes de (1, 3): [(2, 3), (0, 3)]
    Custo de (2, 3): 5 
    Menor Custo: (2, 3) = 5
Adjacentes de (2, 3): [(2, 4), (1, 3)]
    Custo de (2, 4): 4 
    Menor Custo: (2, 4) = 4
Adjacentes de (2, 4): [(1, 4), (2, 3)]
    Custo de (1, 4): 5 
    Menor Custo: (1, 4) = 5
Adjacentes de (1, 4): [(0, 4), (2, 4)]
    Custo de (0, 4): 6 
    Menor Custo: (0, 4) = 6
Adjacentes de (0, 4): [(0, 5), (1, 4)]
    Custo de (0, 5): 5 
    Menor Custo: (0, 5) = 5
Adjacentes de (0, 5): [(1, 5), (0, 6), (0, 4)]
    Custo de (1, 5): 4 
    Custo de (0, 6): 4 
    Menor Custo: (1, 5) = 4
Adjacentes de (1, 5): [(2, 5), (0,